# Project I: Linear Systems and Tridiagonal Matrices

This notebook introduces the fundamental concepts of linear systems in computational physics.

## Overview

After discretization of continuous models (ODEs, PDEs), we often obtain linear systems of the form:

$$A\mathbf{x} = \mathbf{b}$$

where:
- $A \in \mathbb{R}^{n \times n}$ is the coefficient matrix
- $\mathbf{x} \in \mathbb{R}^n$ is the unknown solution vector
- $\mathbf{b} \in \mathbb{R}^n$ is the right-hand side vector

**Key Questions:**
1. Does a solution exist?
2. Is it unique?
3. How sensitive is it to perturbations in data?
4. How can we compute it efficiently?

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../src')

from linear_systems import (
    create_random_system,
    compute_residual,
    build_tridiagonal,
    build_discrete_laplacian_1d
)

# Set random seed for reproducibility
np.random.seed(42)

print("NumPy version:", np.__version__)
print("Setup complete!")

NumPy version: 2.3.4
Setup complete!


## 1. Conditioning of Linear Systems

The **condition number** $\kappa(A) = \|A\| \cdot \|A^{-1}\|$ measures how sensitive the solution is to perturbations in the input data.

- **Well-conditioned**: $\kappa(A) \approx 1$ — small changes in data lead to small changes in solution
- **Ill-conditioned**: $\kappa(A) \gg 1$ — small changes in data can cause large changes in solution

In [11]:
# Create and analyze random systems
n = 50

# Well-conditioned system
A_well = np.eye(n) + 0.1 * np.random.randn(n, n)
b_well = np.random.randn(n)

# Solve the system
x_well = np.linalg.solve(A_well, b_well)
residual_well = compute_residual(A_well, x_well, b_well)
cond_well = np.linalg.cond(A_well)

print("Well-Conditioned System:")
print(f"  Condition number: {cond_well:.2e}")
print(f"  Residual: {residual_well:.2e}")

# Perturb the right-hand side
perturbation = 1e-6 * np.random.randn(n)
b_perturbed = b_well + perturbation
x_perturbed = np.linalg.solve(A_well, b_perturbed)

relative_change_b = np.linalg.norm(perturbation) / np.linalg.norm(b_well)
relative_change_x = np.linalg.norm(x_perturbed - x_well) / np.linalg.norm(x_well)

print(f"  Relative change in b: {relative_change_b:.2e}")
print(f"  Relative change in x: {relative_change_x:.2e}")
print(f"  Amplification factor: {relative_change_x / relative_change_b:.2f}")

Well-Conditioned System:
  Condition number: 7.68e+00
  Residual: 2.53e-15
  Relative change in b: 9.47e-07
  Relative change in x: 9.22e-07
  Amplification factor: 0.97


## 2. Tridiagonal Matrices

A tridiagonal matrix has non-zero elements only on:
- The main diagonal
- The subdiagonal (below main diagonal)
- The superdiagonal (above main diagonal)

$$A = \begin{pmatrix}
d_1 & o_1 & 0 & \cdots & 0 \\
u_1 & d_2 & o_2 & \ddots & \vdots \\
0 & u_2 & d_3 & \ddots & 0 \\
\vdots & \ddots & \ddots & \ddots & o_{n-1} \\
0 & \cdots & 0 & u_{n-1} & d_n
\end{pmatrix}$$

These matrices arise naturally in 1D discretizations.

In [12]:
# Build a simple tridiagonal matrix
n = 5
d = np.array([2.0, 2.0, 2.0, 2.0, 2.0])
u = np.array([-1.0, -1.0, -1.0, -1.0])
o = np.array([-1.0, -1.0, -1.0, -1.0])

A_tri = build_tridiagonal(d, u, o)

print("5×5 Tridiagonal Matrix:")
print(A_tri)
print(f"\nNumber of non-zero elements: {np.count_nonzero(A_tri)}")
print(f"Total elements: {n**2}")
print(f"Sparsity: {np.count_nonzero(A_tri) / n**2 * 100:.1f}% non-zero")

5×5 Tridiagonal Matrix:
[[ 2. -1.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  2.]]

Number of non-zero elements: 13
Total elements: 25
Sparsity: 52.0% non-zero


## 3. The Discrete 1D Laplacian

The discrete second derivative operator on the interval $[0, 1]$ with $n$ interior points:

$$\frac{d^2u}{dx^2} \approx \frac{u_{i-1} - 2u_i + u_{i+1}}{h^2}$$

where $h = \frac{1}{n+1}$ is the grid spacing.

This leads to a tridiagonal matrix with:
- Main diagonal: $-2/h^2$
- Off-diagonals: $1/h^2$

In [13]:
# Build discrete Laplacian for different sizes
n = 10
d, u, o = build_discrete_laplacian_1d(n)
A_laplacian = build_tridiagonal(d, u, o)

print(f"Discrete 1D Laplacian ({n}×{n}):")
print(f"Grid spacing h = {1/(n+1):.4f}")
print(f"\nMatrix structure:")
print(A_laplacian[:5, :5])  # Show top-left corner
print("...")

# Check properties
eigenvalues = np.linalg.eigvalsh(A_laplacian)
print(f"\nMatrix properties:")
print(f"  Symmetric: {np.allclose(A_laplacian, A_laplacian.T)}")
print(f"  All eigenvalues negative: {np.all(eigenvalues < 0)}")
print(f"  Smallest eigenvalue: {eigenvalues[0]:.2f}")
print(f"  Largest eigenvalue: {eigenvalues[-1]:.2f}")
print(f"  Condition number: {np.linalg.cond(A_laplacian):.2e}")

Discrete 1D Laplacian (10×10):
Grid spacing h = 0.0909

Matrix structure:
[[-242.  121.    0.    0.    0.]
 [ 121. -242.  121.    0.    0.]
 [   0.  121. -242.  121.    0.]
 [   0.    0.  121. -242.  121.]
 [   0.    0.    0.  121. -242.]]
...

Matrix properties:
  Symmetric: True
  All eigenvalues negative: True
  Smallest eigenvalue: -474.20
  Largest eigenvalue: -9.80
  Condition number: 4.84e+01


## Next Steps

In the following notebooks, we will:
1. Implement the Thomas algorithm for efficient tridiagonal solving
2. Compare performance with general-purpose solvers
3. Analyze accuracy and stability
4. Apply to simple physical problems